In [1]:
import numpy as np
import pandas as pd

### Load Data Set and EDA

In [2]:
#labeled data http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/

# Reading in and parsing data
raw_data = open('SMSSpamCollection.txt', 'r')
sms_data = []
for line in raw_data:
    split_line = line.split("\t")
    sms_data.append(split_line)

# Splitting data into messages and labels and training and test
sms_data = np.array(sms_data)
X = sms_data[:, 1]
y = sms_data[:, 0]

In [3]:
X

array([ 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...\n',
       'Ok lar... Joking wif u oni...\n',
       "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's\n",
       ..., 'Pity, * was in mood for that. So...any other suggestions?\n',
       "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free\n",
       'Rofl. Its true to its name\n'],
      dtype='|S911')

In [4]:
print sms_data.shape

(5574, 2)


### Build a Linear SVC model

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

#Build vectorized representation of data
vectorizer = TfidfVectorizer()
vectorized_text = vectorizer.fit_transform(X)

text_clf = LinearSVC()
text_clf = text_clf.fit(vectorized_text, y)

print text_clf.predict(vectorizer.transform(["""XXXMobileMovieClub: To use your credit, 
click the WAP link in the next txt message or click here>> 
http://wap. xxxmobilemovieclub.com?n=QJKGIGHJJGCBL"""]))

['spam']


/usr/local/Cellar/python/2.7.12_2/Frameworks/Python.framework/Versions/2.7/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-09-22 16:30:08.675746. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


### Cross - Validation

In [6]:
from sklearn.model_selection import cross_val_score

cross_score = cross_val_score(text_clf, vectorized_text, y, cv=10)
print cross_score
print "mean:",np.mean(cross_score)

[ 0.99103943  0.97491039  0.98387097  0.9874552   0.97849462  0.98387097
  0.98387097  0.98201439  0.98561151  0.98561151]
mean: 0.983674995487


### Converting to CoreML '.mlmodel'

In [11]:
import coremltools

#convert to coreml model
coreml_model = coremltools.converters.sklearn.convert(text_clf, "message", "spam_or_not")

#set parameters of the model
coreml_model.short_description = "Classify whether message is spam or not"
coreml_model.input_description["message"] = "TFIDF of message to be classified"
coreml_model.output_description["spam_or_not"] = "Whether message is spam or not"

coreml_model.save("SpamMessageClassifier.mlmodel")

/usr/local/Cellar/python/2.7.12_2/Frameworks/Python.framework/Versions/2.7/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-09-22 16:31:39.348827. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


### Saving the list of words for TF-IDF

In [8]:
with open("wordlist.txt", "w") as fp:
    for word in vectorizer.get_feature_names():
        fp.write(word.encode('utf-8') + '\n')
    

/usr/local/Cellar/python/2.7.12_2/Frameworks/Python.framework/Versions/2.7/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-09-22 16:29:07.871627. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)
